<a href="https://colab.research.google.com/github/subhra004/End-to-end-Medical-Chatbot-genAI/blob/main/Medical_RAG_Chatbot_using_Biomistral_and_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain sentence-transformers chromadb llama-cpp-python langchain_community pypdf

In [ ]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS, Chroma
from langchain_community.llms import LlamaCpp
from langchain.chains import RetrievalQA, LLMChain

In [ ]:
import pathlib
import textwrap
from IPython.display import display
from IPython.display import Markdown



def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
# Used to securely store your API key
from google.colab import userdata

In [ ]:
# Or use `os.getenv('HUGGINGFACEHUB_API_TOKEN')` to fetch an environment variable.
import os
from getpass import getpass

HUGGINGFACEHUB_API_TOKEN = userdata.get("HUGGINGFACEHUB_API_TOKEN")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "HUGGINGFACEHUB_API_TOKEN"

In [ ]:
loader = PyPDFDirectoryLoader("/content/healthyheart.pdf")
docs = loader.load()

In [ ]:
!pip install langchain-community pymupdf pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 66.1 MB/s eta 0:00:00


In [ ]:
from google.colab import files

uploaded_files = files.upload()  # This allows you to upload multiple PDFs


Saving healthyheart.pdf to healthyheart (1).pdf


In [ ]:
import pandas as pd
from langchain_community.document_loaders import PyPDFDirectoryLoader

# Load all PDFs in the uploaded directory
loader = PyPDFDirectoryLoader("/content")
docs = loader.load()

# Convert to DataFrame
data = {"filename": [], "content": []}

for doc in docs:
    data["filename"].append(doc.metadata["source"])  # Get the filename
    data["content"].append(doc.page_content)  # Extracted text

df = pd.DataFrame(data)
print(df.head())  # Show first few rows


                        filename  \
0  /content/healthyheart (1).pdf   
1  /content/healthyheart (1).pdf   
2  /content/healthyheart (1).pdf   
3  /content/healthyheart (1).pdf   
4  /content/healthyheart (1).pdf   

                                             content  
0  YOUR GUIDE TO\nA Healthy Heart\nU.S. DEPARTMEN...  
1  YOUR GUIDE TO\nA Healthy Heart\nU.S. DEPARTMEN...  
2  U.S. DEPARTMENT OF HEALTH AND HUMAN SERVICES\n...  
3  Heart Disease: Why Should You Care? . . . . . ...  
4  iv\nTaking Charge:  An Action Plan for Heart H...  


In [ ]:
docs

[Document(metadata={'producer': 'Acrobat Distiller 6.0.1 for Macintosh', 'creator': 'QuarkXPress(tm) 6.5', 'creationdate': '2006-02-16T11:30:29-05:00', 'subject': 'Heart disease', 'author': 'NHLBI', 'keywords': 'heart disease, prevention, risk factors, chd, coronary artery disease, corornary heart disease, cad', 'moddate': '2006-02-23T09:58:15-05:00', 'title': 'Your Guide to A Healthy Heart', 'source': '/content/healthyheart (1).pdf', 'total_pages': 95, 'page': 0, 'page_label': '1'}, page_content='YOUR GUIDE TO\nA Healthy Heart\nU.S. DEPARTMENT OF HEALTH AND HUMAN SERVICES\nNational Institutes of Health\nNational Heart, Lung, and Blood Institute'),
 Document(metadata={'producer': 'Acrobat Distiller 6.0.1 for Macintosh', 'creator': 'QuarkXPress(tm) 6.5', 'creationdate': '2006-02-16T11:30:29-05:00', 'subject': 'Heart disease', 'author': 'NHLBI', 'keywords': 'heart disease, prevention, risk factors, chd, coronary artery disease, corornary heart disease, cad', 'moddate': '2006-02-23T09:58:

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = text_splitter.split_documents(docs)

In [ ]:
len(chunks)

1506

In [ ]:
chunks[0]

Document(metadata={'producer': 'Acrobat Distiller 6.0.1 for Macintosh', 'creator': 'QuarkXPress(tm) 6.5', 'creationdate': '2006-02-16T11:30:29-05:00', 'subject': 'Heart disease', 'author': 'NHLBI', 'keywords': 'heart disease, prevention, risk factors, chd, coronary artery disease, corornary heart disease, cad', 'moddate': '2006-02-23T09:58:15-05:00', 'title': 'Your Guide to A Healthy Heart', 'source': '/content/healthyheart (1).pdf', 'total_pages': 95, 'page': 0, 'page_label': '1'}, page_content='YOUR GUIDE TO\nA Healthy Heart\nU.S. DEPARTMENT OF HEALTH AND HUMAN SERVICES\nNational Institutes of Health\nNational Heart, Lung, and Blood Institute')

In [ ]:
chunks[1]

Document(metadata={'producer': 'Acrobat Distiller 6.0.1 for Macintosh', 'creator': 'QuarkXPress(tm) 6.5', 'creationdate': '2006-02-16T11:30:29-05:00', 'subject': 'Heart disease', 'author': 'NHLBI', 'keywords': 'heart disease, prevention, risk factors, chd, coronary artery disease, corornary heart disease, cad', 'moddate': '2006-02-23T09:58:15-05:00', 'title': 'Your Guide to A Healthy Heart', 'source': '/content/healthyheart (1).pdf', 'total_pages': 95, 'page': 1, 'page_label': '2'}, page_content='YOUR GUIDE TO\nA Healthy Heart\nU.S. DEPARTMENT OF HEALTH AND HUMAN SERVICES\nNational Institutes of Health\nNational Heart, Lung, and Blood Institute\nNIH Publication No. 06-5269\nDecember 2005')

In [ ]:
chunks[2]

Document(metadata={'producer': 'Acrobat Distiller 6.0.1 for Macintosh', 'creator': 'QuarkXPress(tm) 6.5', 'creationdate': '2006-02-16T11:30:29-05:00', 'subject': 'Heart disease', 'author': 'NHLBI', 'keywords': 'heart disease, prevention, risk factors, chd, coronary artery disease, corornary heart disease, cad', 'moddate': '2006-02-23T09:58:15-05:00', 'title': 'Your Guide to A Healthy Heart', 'source': '/content/healthyheart (1).pdf', 'total_pages': 95, 'page': 2, 'page_label': '3'}, page_content='U.S. DEPARTMENT OF HEALTH AND HUMAN SERVICES\nNational Institutes of Health\nNational Heart, Lung, and Blood Institute\nWritten by: Marian Sandmaier')

In [ ]:
chunks[3]

Document(metadata={'producer': 'Acrobat Distiller 6.0.1 for Macintosh', 'creator': 'QuarkXPress(tm) 6.5', 'creationdate': '2006-02-16T11:30:29-05:00', 'subject': 'Heart disease', 'author': 'NHLBI', 'keywords': 'heart disease, prevention, risk factors, chd, coronary artery disease, corornary heart disease, cad', 'moddate': '2006-02-23T09:58:15-05:00', 'title': 'Your Guide to A Healthy Heart', 'source': '/content/healthyheart (1).pdf', 'total_pages': 95, 'page': 3, 'page_label': '4'}, page_content='Heart Disease: Why Should You Care? . . . . . . . . . . . . . . . . . . . . . . . . . . . . 1\nWhat You Need To Know About Heart Disease . . . . . . . . . . . . . . . . . . . . . . 3\nWhat Is Heart Disease?. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 3')

In [ ]:
chunks[4]

Document(metadata={'producer': 'Acrobat Distiller 6.0.1 for Macintosh', 'creator': 'QuarkXPress(tm) 6.5', 'creationdate': '2006-02-16T11:30:29-05:00', 'subject': 'Heart disease', 'author': 'NHLBI', 'keywords': 'heart disease, prevention, risk factors, chd, coronary artery disease, corornary heart disease, cad', 'moddate': '2006-02-23T09:58:15-05:00', 'title': 'Your Guide to A Healthy Heart', 'source': '/content/healthyheart (1).pdf', 'total_pages': 95, 'page': 3, 'page_label': '4'}, page_content='Who Is at Risk? . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 4\nHow Risk Works . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 4')

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en-v1.5")

<ipython-input-20-11fdde04d023>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en-v1.5")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warni

In [ ]:
# Make sure you have installed the necessary package:
!pip install langchain_community

# Import the HuggingFaceEmbeddings class:
from langchain_community.embeddings import HuggingFaceEmbeddings

# Now you can create an instance of it:
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en-v1.5")

In [ ]:
vectorstore = Chroma.from_documents(chunks, embeddings)

In [ ]:
vectorstore

In [ ]:
query = "who is at risk of heart disease"
search = vectorstore.similarity_search(query)

In [ ]:
to_markdown(search[0].page_content)

> The first step toward heart health is becoming aware of your 
> own personal risk for heart disease. Some risks, such as smoking
> cigarettes or being overweight, are obvious: All of us know whether
> we smoke or whether we need to lose a few pounds. But other risk

In [ ]:
retriever = vectorstore.as_retriever(
    search_kwargs={'k': 5}
)

In [ ]:
retriever.get_relevant_documents(query)

<ipython-input-27-162c3489abc9>:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents(query)


[Document(metadata={'author': 'NHLBI', 'creationdate': '2006-02-16T11:30:29-05:00', 'creator': 'QuarkXPress(tm) 6.5', 'keywords': 'heart disease, prevention, risk factors, chd, coronary artery disease, corornary heart disease, cad', 'moddate': '2006-02-23T09:58:15-05:00', 'page': 9, 'page_label': '10', 'producer': 'Acrobat Distiller 6.0.1 for Macintosh', 'source': '/content/healthyheart.pdf', 'subject': 'Heart disease', 'title': 'Your Guide to A Healthy Heart', 'total_pages': 95}, page_content='The first step toward heart health is becoming aware of your \nown personal risk for heart disease. Some risks, such as smoking\ncigarettes or being overweight, are obvious: All of us know whether\nwe smoke or whether we need to lose a few pounds. But other risk'),
 Document(metadata={'author': 'NHLBI', 'creationdate': '2006-02-16T11:30:29-05:00', 'creator': 'QuarkXPress(tm) 6.5', 'keywords': 'heart disease, prevention, risk factors, chd, coronary artery disease, corornary heart disease, cad', '

In [ ]:
#connect to google drive
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
llm = LlamaCpp(
    model_path= "/content/drive/MyDrive/Colab Notebooks/BioMistral-7B.Q4_K_M.gguf",
    temperature=0.3,
    max_tokens=2048,
    top_p=1)

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from /content/drive/MyDrive/Colab Notebooks/BioMistral-7B.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = hub
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llam

In [ ]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import ChatPromptTemplate

In [ ]:
template = """
<|context|>
You are an AI assistant that follows instruction extremely well.
Please be truthful and give direct answers
</s>
<|user|>
{query}
</s>
 <|assistant|>
"""

In [ ]:
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
rag_chain = (
    {"context": retriever,  "query": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
response = rag_chain.invoke("what disease affect the heart?")

llama_perf_context_print:        load time =   27772.94 ms
llama_perf_context_print: prompt eval time =   27772.70 ms /    58 tokens (  478.84 ms per token,     2.09 tokens per second)
llama_perf_context_print:        eval time =   62341.67 ms /    70 runs   (  890.60 ms per token,     1.12 tokens per second)
llama_perf_context_print:       total time =   90209.85 ms /   128 tokens


In [ ]:
to_markdown(response)

> The heart is affected by many diseases. Some of the most common heart diseases include coronary artery disease, hypertension, arrhythmia, and heart failure. It can also be affected by congenital heart defects, valvular heart disease, cardiomyopathies, endocarditis, and myocarditis.

In [ ]:
import sys

while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue
  result = rag_chain.invoke(user_input)
  print("Answer: ",result)

Llama.generate: 41 prefix-match hit, remaining 15 prompt tokens to eval
llama_perf_context_print:        load time =   27772.94 ms
llama_perf_context_print: prompt eval time =    6151.99 ms /    15 tokens (  410.13 ms per token,     2.44 tokens per second)
llama_perf_context_print:        eval time =   59858.95 ms /    70 runs   (  855.13 ms per token,     1.17 tokens per second)
llama_perf_context_print:       total time =   66105.62 ms /    85 tokens


Answer:  Heart disease is a condition in which the heart and blood vessels are not functioning well. It can lead to serious health problems such as stroke, heart attack, and heart failure. There are many different types of heart diseases, including coronary artery disease, arrhythmia, congenital heart defects, and valvular heart disease.
